#Set-Up

In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [2]:
npv = [5.6, 2.7, 3.9, 6.8, 7.7, 5.1, 2.8, 3.5, 4.5, 5.1, 6.1, 9.5, 7.2, 4.9, 3.1, 3.6, 1.55, 0.29, 5.2, 5.9, 6.1, 3.5, 4.5, 5.2, 2.1, 3.7]


In [3]:
len(npv)

26

#Capital Budgeting Problem (Base)

In [4]:
#Capital Budgeting Base Problem Data
cost = [5, 2.4, 3.5, 5.9, 6.9, 4.4, 2.5, 3.1, 3.2, 4, 5.1, 6.1, 4.4, 4.4, 2.5, 3.1, 2.15, 1.57, 2.3, 5.4, 5, 1.9, 2.5, 4, 1.5, 3]
npv = [5.6, 2.7, 3.9, 6.8, 7.7, 5.1, 2.8, 3.5, 4.5, 5.1, 6.1, 9.5, 7.2, 4.9, 3.1, 3.6, 1.55, 0.29, 5.2, 5.9, 6.1, 3.5, 4.5, 5.2, 2.1, 3.7]
budget = 50
n = len(cost)

#define the concrete model
model = ConcreteModel()

#DVs
model.x = Var(range(n), domain = Binary) #NOTE: This is where we restrict the decision variables to be binary (could also put things like NonNegativeIntegers)

#budget constraint
model.BudgetConstraint = Constraint(expr = sum(cost[i]*model.x[i] for i in range(n)) <= budget)

#objective
model.Objective = Objective(expr = sum(npv[i]*model.x[i] for i in range(n)), sense = maximize)

In [5]:
#solve the model
opt = SolverFactory('cbc')
opt.options['seconds'] = 5 #specifies the time limit (in seconds)
opt.options['ratioGap'] = .01 #specifies the optimality gap tolerance (.01 means alg can stop if guarenteed within <1% of optimal obj)
results = opt.solve(model, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Jun  7 2023 

command line - /content/bin/cbc -seconds 5 -ratioGap 0.01 -printingOptions all -import /tmp/tmpg2agk_oy.pyomo.lp -stat=1 -solve -solu /tmp/tmpg2agk_oy.pyomo.soln (default strategy 1)
seconds was changed from 1e+100 to 5
ratioGap was changed from 0 to 0.01
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 1 (0) rows, 26 (0) columns and 26 (0) elements
Statistics for presolved model
Original problem has 26 integers (26 of which binary)
==== 0 zero objective 21 different
==== absolute objective values 21 different
==== for integers 0 zero objective 21 different
==== for integers absolute objective values 21 different
===== end objective counts


Problem has 1 rows, 26 columns (26 with objective) and 26 elements
There are 26 singletons with objective 
Column breakdown

In [6]:
#print out solution
print("invest in options: ")
for i in range(n):
    if model.x[i]() == 1:
        print(i)

print("Total NPV:", model.Objective())
print("Total Cost:", model.BudgetConstraint())

invest in options: 
5
8
9
10
11
12
14
18
20
21
22
23
24
25
Total NPV: 70.9
Total Cost: 49.9


In [7]:
#problem 1 - constraint 1
#constraint: at most 2 investments from numbers 20-26
model.InvestmentLimit=Constraint(expr=sum(model.x[i] for i in range (19,26))<=2)
model.pprint()

#problem 2- constraint 2
#if you invest in number 11, you must also invest in 12
#model.InvestmentLimit=Constraint(expr=model.x[10]<=model.x[11])

#problem 3
#must invest in 6 to be able to invest in any of 7-10
model.InvestmentLimit = ConstraintList()
for i in range(6, 10):
    model.InvestmentLimit.add(expr=model.x[i]<= model.x[5])

#problem 4 guarantee global optimality using by setting ratio gap to 0
#solve the model
opt = SolverFactory('cbc')
opt.options['seconds'] = 5 #specifies the time limit (in seconds)
opt.options['ratioGap'] = 0 #specifies the optimality gap tolerance (.01 means alg can stop if guarenteed within <1% of optimal obj)
results = opt.solve(model, tee=True)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


1 Var Declarations
    x : Size=26, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   0.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   0.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   0.0 :     1 : False : False : Binary
          5 :     0 :   1.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :   1.0 :     1 : False : False : Binary
          9 :     0 :   1.0 :     1 : False : False : Binary
         10 :     0 :   1.0 :     1 : False : False : Binary
         11 :     0 :   1.0 :     1 : False : False : Binary
         12 :     0 :   1.0 :     1 : False : False : Binary
         